# TF-IDF with Greek

Use this notebook to experiment with calculating TF-IDF with Greek. You won't be able to use scikit-learn's `"english"` stop words, and in general you will have to be in charge of a much larger part of the pipeline.

But this could be a useful exercise for those of you interested in the additional complexity of working with Greek!

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.read_pickle("./greek-tragedy-by-line.pickle")

df[df['speaker'].str.lower() == 'chorus']